# Semantic chunking

Next, we are going to try using a semantic text splitter to chunk the text to blocks of around 512 tokens.

## 1. Setup

We will use semantic-text-splitter from PyPI and the bert base uncased tokenizer from HuggingFace.

In [1]:
# Change working directory to parent so we can import as we would from __main__.py
print(f'Working directory: ', end = '')
%cd ..

# Standard ports
import time

# PyPI imports
import h5py
from semantic_text_splitter import TextSplitter
from tokenizers import Tokenizer

# Internal imports
import configuration as config

Working directory: /mnt/arkk/opensearch/semantic_search


In [2]:
input_file=f'{config.DATA_PATH}/wikipedia/{config.BATCHED_TEXT}'

tokenizer_name='bert-base-uncased'
max_tokens=512

tokenizer=Tokenizer.from_pretrained(tokenizer_name)
splitter=TextSplitter.from_huggingface_tokenizer(tokenizer, max_tokens)

## 2. Load data

Load up the first batch from the data extractor.

In [3]:
input_data=h5py.File(input_file, 'r')
batch=input_data['batches/0']

sample_text=' '.join(batch[0].decode('utf-8').split(' ')[:100])

print(f'First batch contains {len(batch)} texts')
print(f"First text:\n{sample_text}")

First batch contains 10000 texts
First text:
Hercule Poirot (UK: /ˈɛərkjuːl ˈpwɑːroʊ/, US: /hɜːrˈkjuːl pwɑːˈroʊ/) is a fictional Belgian detective created by British writer Agatha Christie. Poirot is one of Christie's most famous and long-running characters, appearing in 33 novels, two plays (Black Coffee and Alibi), and 51 short stories published between 1920 and 1975. Poirot has been portrayed on radio, in film and on television by various actors, including Austin Trevor, John Moffatt, Albert Finney, Peter Ustinov, Ian Holm, Tony Randall, Alfred Molina, Orson Welles, David Suchet, Kenneth Branagh, and John Malkovich. Poirot's name was derived from two other fictional detectives of the time: Marie Belloc


## 3. Test semantic chunking
Test split the first text from the first batch:

In [8]:
chunks=splitter.chunks(batch[0].decode('utf-8'))
print(f'Chunks are: {type(chunks)}')
print(f'Have {len(chunks)} chunks')

for chunk in chunks:
    chunk_start=' '.join(chunk.split(' ')[:25])
    chunk_end=' '.join(chunk.split(' ')[-25:])
    print(f'\n{chunk_start} ... {chunk_end}')

Chunks are: <class 'list'>
Have 27 chunks

Hercule Poirot (UK: /ˈɛərkjuːl ˈpwɑːroʊ/, US: /hɜːrˈkjuːl pwɑːˈroʊ/) is a fictional Belgian detective created by British writer Agatha Christie. Poirot is one of Christie's most ... years. Christie's Poirot was clearly the result of her early development of the detective in her first book, written in 1916 and published in 1920.

The large number of refugees in the country who had fled the German invasion of Belgium in August to November 1914 served as a plausible ... is not mentioned, suggesting it may have been a temporary wartime injury. (In Curtain, Poirot admits he was wounded when he first came to England.)

Poirot has green eyes that are repeatedly described as shining "like a cat's" when he is struck by a clever idea, and dark hair, which ... the dark throughout the climax. This aspect of Poirot is less evident in the later novels, partly because there is rarely a narrator to mislead.

In Murder on the Links, still largely dependent on 

OK, pretty good. We didn't break up any sentences. Some chunks start with a pronoun, so they would be a little unclear to read in isolation. But this approach is obviously much better than taking chunks by word count. Might want to think about a clever way to filter out chunks that only contain references.

It feels snappy too - let's time chunking a few batches and see what we are working with.

## 4. Semantic text chunking rate

In [10]:
%%time

# Number of batches to time chunking for
num_batches=1

# Holder to collect chunking times
chunking_times=[]

for i in range(num_batches):

    # Start the timer
    start_time=time.time()

    # Get the text batch
    batch=input_data[f'batches/{i}']

    # Chunk the records from the batch
    for record in batch:
        chunks=splitter.chunks(record.decode('utf-8'))

    # Stop the timer
    dT=time.time() - start_time
    chunking_times.append(dT)

mean_chunking_time=sum(chunking_times)/len(chunking_times)
print(f'Mean chunking time {mean_chunking_time::.1f} seconds per record\n')

Mean chunking time 169.15254759788513 seconds per record
CPU times: user 2min 49s, sys: 7.87 ms, total: 2min 49s
Wall time: 2min 49s


Ok here's the estimate: we have 688 batches so about 32 hours to chunk the whole thing.

If we parallelize it across our CPU cores, we should be able to get it down to under two hours.